<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Getting started with OGGM: Ötztal case study

The OGGM workflow is best explained with an example. In this example we will show how to apply the standard [OGGM workflow](http://oggm.readthedocs.io/en/latest/introduction.html) to a list of glaciers.

We are going to use the list of glacier we use for testing the oggm codebase. The test files are located in a dedicated online repository, [oggm-sample-data](https://github.com/OGGM/oggm-sample-data).

## Input data

In the `test-workflow` directory you can have a look at the various files we will need. oggm also needs them for testing, so they are automatically available to everybody with a simple mechanism:

In [ ]:
import oggm
from oggm import cfg
from oggm import utils
cfg.initialize()
srtm_f = utils.get_demo_file('srtm_oetztal.tif')
rgi_f = utils.get_demo_file('rgi_oetztal.shp')
print(srtm_f)

The very first time that you make a call to `get_demo_file()`, oggm will create a hidden `.oggm` directory in your home folder$^*$ and download the demo files in it.

<sub>*: this path might vary depending on your platform, see python's [expanduser](https://docs.python.org/3.5/library/os.path.html#os.path.expanduser)</sub>

### DEM and glacier outlines

The data directory contains a subset of the RGI (V5) for the Ötztal:

In [ ]:
import geopandas as gpd
rgi_shp = gpd.read_file(rgi_f).set_index('RGIId')

We'll have a look at it, but first we will need to make some imports and set some defaults:

In [ ]:
# Plot defaults
%matplotlib inline
import matplotlib.pyplot as plt
# Packages
import os
import numpy as np
import xarray as xr
import shapely.geometry as shpg
plt.rcParams['figure.figsize'] = (8, 8)  # Default plot size

Plot the glaciers of the Ötztal case study:

In [ ]:
rgi_shp.plot(edgecolor='black');

### Calibration / validation data

These 20 glaciers were selected because they have either mass-balance data (WGMS) or total volume information (GlaThiDa). These data are required for calibration/validation and are available automatically in OGGM.

### Climate data

For this test case we use HISTALP data (which goes back further in time than CRU), stored in the NetCDF format. The resolution of HISTALP (5 minutes of arc) is relatively high, but some kind of downscaling will be necessary to compute the mass-balance at the glacier scale.

We can plot a timeseries of the data, for example for the grid point (3, 3):

In [ ]:
fig = plt.figure(figsize=(9, 3))
with xr.open_dataset(utils.get_demo_file('HISTALP_oetztal.nc')) as ds:
    ds.temp[:, 3, 3].resample(time="AS").mean().plot()
    plt.title('HISTALP annual temperature (°C)');

## Setting up an OGGM run

OGGM parameters are gathered in a configuration file. The [default file](https://github.com/OGGM/oggm/blob/master/oggm/params.cfg) is shipped with the code. It is used to initialize the configuration module:

In [ ]:
from oggm import cfg
from oggm import workflow
cfg.initialize()  # read the default parameter file

For example, the `cfg` module has a global variable `PATHS` (a dictionary) storing the file paths to the data and working directories:

In [ ]:
cfg.PATHS

The path to the working directory and the input data files are missing. Let's set them so that the oggm modules know where to look for them (the default would be to download them automatically, which we would like to avoid for this example):

In [ ]:
cfg.PATHS['working_dir'] = os.path.expanduser('~/OGGM_Getting_Started_wd')

In [ ]:
cfg.PATHS['dem_file'] = utils.get_demo_file('srtm_oetztal.tif')
cfg.PATHS['climate_file'] = utils.get_demo_file('HISTALP_oetztal.nc')
cfg.set_intersects_db(utils.get_demo_file('rgi_intersect_oetztal.shp'))

We will set the "border" option to a larger value, since we will do some dynamical simulations ("border" decides on the number of DEM grid points we'd like to add to each side of the glacier for the local map: the larger the glacier will grow, the larger border should be):

In [ ]:
cfg.PARAMS['border'] = 80

We keep the other parameters to their default values, for example the precipitation scaling factor:

In [ ]:
cfg.PARAMS['prcp_scaling_factor']

## Glacier working directories

An OGGM "run" is made of several successive tasks to be applied on each glacier. Because these tasks can be computationally expensive they are split in smaller tasks, each of them storing their results in a [glacier directory](http://oggm.readthedocs.org/en/latest/glacierdir.html).

The very first task of an OGGM run is always `init_glacier_regions`:

In [ ]:
# Read in the RGI file
import geopandas as gpd
rgi_file = utils.get_demo_file('rgi_oetztal.shp')
rgidf = gpd.read_file(rgi_file)
# Initialise directories
# reset=True will ask for confirmation if the directories are already present: 
# this is very useful if you don't want to loose hours of computations because of a command gone wrong
gdirs = oggm.workflow.init_glacier_regions(rgidf, reset=True, force=True)

Note that if I run `init_glacier_regions` a second time without `reset=True`, nothing special happens. The directories will not be overwritten, just "re-opened":

In [ ]:
gdirs = workflow.init_glacier_regions(rgidf)

Now what is the variable `gdirs`? It is a list of 19 [GlacierDirectory](http://oggm.readthedocs.org/en/latest/generated/oggm.GlacierDirectory.html#oggm.GlacierDirectory) objects. They are here to help us to handle data input/output and to store several glacier properties. Here are some examples: 

In [ ]:
gdir = gdirs[13]
gdir

`gdir` provides a `get_filepath` function which gives access to the data files present in the directory:

In [ ]:
gdir.get_filepath('dem')

`dem.tif` is a local digital elevation map with a spatial resolution chosen by OGGM as a function of the glacier size. These [GlacierDirectory](http://oggm.readthedocs.org/en/latest/generated/oggm.GlacierDirectory.html#oggm.GlacierDirectory) objects are going to be the input of almost every OGGM task.

This data model has been chosen so that even complex functions requires serval input data can be called with one single argument: 

In [ ]:
from oggm import graphics
graphics.plot_googlemap(gdir)

## OGGM tasks

The workflow of OGGM is oriented around the concept of "[tasks](http://oggm.readthedocs.org/en/latest/api.html#entity-tasks)". There are two different types:

**Entity Task**:
  Standalone operations to be realized on one single glacier entity,
  independently from the others. The majority of OGGM
  tasks are entity tasks. They are parallelisable.

**Global Task**:
  tasks which require to work on several glacier entities
  at the same time. Model parameter calibration or interpolation of degree day factors belong to
  this type of task. They are not parallelisable.
  
OGGM implements a simple mechanism to run a specific task on a list of `GlacierDir` objects (here, the function `glacier_masks()` from the module `oggm.prepro.gis`):

In [ ]:
from oggm import tasks

In [ ]:
# run the glacier_masks task on all gdirs
workflow.execute_entity_task(tasks.glacier_masks, gdirs)

We just computed gridded boolean [masks](http://oggm.readthedocs.org/en/latest/generated/oggm.tasks.glacier_masks.html#oggm.tasks.glacier_masks) out of the RGI outlines.

It is also possible to apply several tasks sequentially:

In [ ]:
list_talks = [
         tasks.compute_centerlines,
         tasks.initialize_flowlines,
         tasks.catchment_area,
         tasks.catchment_width_geom,
         tasks.catchment_width_correction,
         tasks.compute_downstream_line,
         tasks.compute_downstream_bedshape
         ]
for task in list_talks:
    workflow.execute_entity_task(task, gdirs)

The function `execute_task` can run a task on different glaciers at the same time, if the `use_multiprocessing` option is set to `True` in the configuration file. 

With all these tasks we just computed the glacier flowlines and their width:

In [ ]:
graphics.plot_catchment_width(gdir, corrected=True)

### Global tasks, climate tasks

We will go into more detail about tasks in the documentation. For now, we will use the helper function:

In [ ]:
# We calibrate tstars ourselves
cfg.PARAMS['run_mb_calibration'] = True
workflow.climate_tasks(gdirs)

We just read the climate data, "downscaled" it to each glacier, computed possible $\mu^*$ for the reference glaciers, picked the best one, interpolated the corresponding $t^*$ to glaciers without mass-balance observations, computed the mass-balance sensitivity $\mu$ for all glaciers and finally computed the mass-balance at equilibrium (the "apparent mb" in Farinotti et al., 2009).

## Inversion

This is where things become a bit more complicated. The inversion is already fully automated in OGGM, but there is not yet a generally accepted way to calibrate it. For this tutorial we will try to explain in more detail what is happening.

### Defaut parameters

Let's start with the default, which is to use the standard ice creep parameter A and no sliding:

In [ ]:
list_talks = [
         tasks.prepare_for_inversion,  # This is a preprocessing task
         tasks.volume_inversion,  # This does the actual job
         tasks.filter_inversion_output  # This smoothes the thicknesses at the tongue a little
         ]
for task in list_talks:
    workflow.execute_entity_task(task, gdirs)

Let's what we have:

In [ ]:
# Select HEF out of all glaciers
gdir_hef = [gd for gd in gdirs if (gd.rgi_id == 'RGI50-11.00897')][0]
graphics.plot_inversion(gdir_hef)

The computed volume of Hintereisferner is 0.77 km$^{3}$. According to Fisher et al, 2013, this volume should be 0.573 km$^{3}$, which is a bit less then computed. This is not too surprising, since we use a default setting and no basal sliding, all this likely to overestimate the thickness.

### Optimize the parameters 

OGGM currently has a built-in way to optimize the ice dynamics parameters `A` and `fs` (the later being zero per default) based on GlaThiDa data (http://www.gtn-g.ch/data_catalogue_glathida/). We calibrate A so that the total volume RMSD with respect to the total volume of the reference glaciers is smallest:

In [ ]:
cfg.PARAMS['optimize_inversion_params'] = True
tasks.optimize_inversion_params(gdirs)  # This time we optimise the parameters

This converges towards a creep parameter `A` almost 5 times larger than the default one. Let's have a look at how we perform:

In [ ]:
import pandas as pd
fpath = os.path.join(cfg.PATHS['working_dir'], 'inversion_optim_results.csv')
df = pd.read_csv(fpath, index_col=0)
df['ref_thick'] = df['ref_volume_km3'] / df['ref_area_km2'] * 1e3
df['oggm_thick'] = df['oggm_volume_km3'] / df['ref_area_km2'] * 1e3
df['vas_thick'] = df['vas_volume_km3'] / df['ref_area_km2'] * 1e3

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.scatter(df['ref_thick'], df['oggm_thick'], s=100)
ax1.set_title('OGGM RMSD: {:.2f}'.format(oggm.utils.rmsd(df['ref_thick'], df['oggm_thick'])))
ax1.set_xlabel('Ref thickness')
ax1.set_ylabel('OGGM thickness')
ax1.plot([0, 100], [0, 100], '.:k', zorder=0);
ax1.set_xlim([0, 100]), ax1.set_ylim([0, 100]);
ax2.scatter(df['ref_thick'], df['vas_thick'], s=100)
ax2.set_title('Volume-Area RMSD: {:.2f}'.format(oggm.utils.rmsd(df['ref_thick'], df['vas_thick'])))
ax2.set_xlabel('Ref thickness')
ax2.set_ylabel('VAS thickness')
ax2.plot([0, 100], [0, 100], '.:k', zorder=0);
ax2.set_xlim([0, 100]), ax2.set_ylim([0, 100]);

This is not bad, but we have to keep a few things in mind:
- the calibration data from GlaThiDa also contains non-negligible errors, mostly due to three factors: the date of measurement doesn't necessaily correspond to the RGI outlines, the total volume is a subjective value obtained from point GPR measurements, and these points also are uncertain
- the OGGM inversion calibration is quite sensitive to the parameter A, as we show below.

All this calls for a better calibration of the model -- and is certainly something we want to tackle in the near future.

### Inversion model sensitivity 

How sensitive is the inversion to changes in the A parameter?

In [ ]:
factor = np.linspace(0.1, 10, 30)
thick = factor*0
for i, f in enumerate(factor):
    vol_m3, area_m3 = tasks.volume_inversion(gdir_hef, glen_a=cfg.A*f, print_log=False)
    thick[i] = vol_m3/area_m3
plt.figure(figsize=(6, 4))
plt.plot(factor, thick);
plt.ylabel('Mean thickness (m)');
plt.xlabel('Multiplier');

The shape of the curve is explained by the physics of ice. As you can see, tuning this parameter is not an easy task.

### Finalize the inversion 

In [ ]:
# For the rest of the examples, we don't use the optimal parameters but the default ones
cfg.PARAMS['optimize_inversion_params'] = False
workflow.execute_entity_task(tasks.volume_inversion, gdirs)
workflow.execute_entity_task(tasks.filter_inversion_output, gdirs)

The results of the inversion (and other useful parameters) can be combined with a utilitary function:

In [ ]:
df = utils.glacier_characteristics(gdirs)
ax = df.plot.scatter(x='rgi_area_km2', y='inv_volume_km3', color='C3')
ax.semilogx(); ax.semilogy();

## Dynamics 

For the dynamics refer to the other notebooks in the same folder!

## Set up a run 

For setting up a real OGGM run with your own data, refer to [the documentation](http://oggm.readthedocs.io/en/latest/run.html)!